In [1]:
%cd ../..

C:\Users\ManuJoseph\OneDrive - Thoucentric\Work\Projects\Playground\AdvancedTimeSeriesForecastingBook\Github\Modern-Time-Series-Forecasting-with-Python-


In [2]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
from tqdm.autonotebook import tqdm
import warnings
from src.utils.general import LogTime
import plotly.express as px
import matplotlib.pyplot as plt
import joblib
from src.transforms.target_transformations import AutoStationaryTransformer
from IPython.display import display, HTML
# %load_ext autoreload
# %autoreload 2
np.random.seed(42)
tqdm.pandas()

C:\Users\MANUJO~1\AppData\Local\Temp/ipykernel_18952/3139195564.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
os.makedirs("imgs/chapter_6", exist_ok=True)
preprocessed = Path("data/london_smart_meters/preprocessed")

## Reading the preprocessed and feature engineered files (Train + Validation)

In [4]:
#Readin the missing value imputed and train test split data
try:
    train_df = pd.read_parquet(preprocessed/"selected_blocks_train_missing_imputed_feature_engg.parquet")
    val_df = pd.read_parquet(preprocessed/"selected_blocks_val_missing_imputed_feature_engg.parquet")
    train_df["type"] = "train"
    val_df["type"] = "val"
    train_df = pd.concat([train_df,val_df])
    del val_df
    display(train_df.head())
except FileNotFoundError:
    display(HTML("""
    <div class="alert alert-block alert-warning">
    <b>Warning!</b> File not found. Please make sure you have run 01-Feature Engineering.ipynb in Chapter06
    </div>
    """))

,timestamp,LCLid,energy_consumption,frequency,series_length,stdorToU,Acorn,Acorn_grouped,file,holidays,...,timestamp_Minute_sin_2,timestamp_Minute_sin_3,timestamp_Minute_sin_4,timestamp_Minute_sin_5,timestamp_Minute_cos_1,timestamp_Minute_cos_2,timestamp_Minute_cos_3,timestamp_Minute_cos_4,timestamp_Minute_cos_5,type
0,2012-01-01 00:00:00,MAC000061,0.114,30min,37872,Std,ACORN-Q,Adversity,block_96,NO_HOLIDAY,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,1.0,1.0,1.0,1.0,train
1,2012-01-01 00:30:00,MAC000061,0.113,30min,37872,Std,ACORN-Q,Adversity,block_96,NO_HOLIDAY,...,-1.133108e-15,2.143751e-15,-2.266215e-15,6.123234e-16,-1.0,1.0,-1.0,1.0,-1.0,train
2,2012-01-01 01:00:00,MAC000061,0.113,30min,37872,Std,ACORN-Q,Adversity,block_96,NO_HOLIDAY,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,1.0,1.0,1.0,1.0,train
3,2012-01-01 01:30:00,MAC000061,0.098,30min,37872,Std,ACORN-Q,Adversity,block_96,NO_HOLIDAY,...,-1.133108e-15,2.143751e-15,-2.266215e-15,6.123234e-16,-1.0,1.0,-1.0,1.0,-1.0,train
4,2012-01-01 02:00:00,MAC000061,0.060,30min,37872,Std,ACORN-Q,Adversity,block_96,NO_HOLIDAY,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,1.0,1.0,1.0,1.0,train


In [5]:
#Takes a while. Snack Break!
transformer_pipelines = {}
for _id in tqdm(train_df["LCLid"].unique()):
    #Initialize the AutoStationaryTransformer with a seasonality period of 48*7
    auto_stationary = AutoStationaryTransformer(seasonal_period=48*7)
    #Creating the timeseries with datetime index
    y = train_df.loc[train_df["LCLid"]==_id, ["energy_consumption","timestamp"]].set_index("timestamp")
    #Fitting and trainsforming the train
    y_stat = auto_stationary.fit_transform(y, freq="30min")
    # Setting the transformerd series back to the dataframe
    train_df.loc[train_df["LCLid"]==_id, "energy_consumption"] = y_stat.values
    #Saving the pipeline
    transformer_pipelines[_id] = auto_stationary

  0%|          | 0/150 [00:00<?, ?it/s]

## Saving the transformed file as well as the transformer pipelines

In [6]:
train_df = train_df.loc[:,["LCLid","timestamp","energy_consumption"]].set_index(["LCLid","timestamp"])
train_df.rename(columns={"energy_consumption":"energy_consumption_auto_stat"}, inplace=True)

In [7]:
train_df.head()

energy_consumption_auto_stat
LCLid     timestamp                                        
MAC000061 2012-01-01 00:00:00                      0.555366
          2012-01-01 00:30:00                      0.560124
          2012-01-01 01:00:00                      0.563143
          2012-01-01 01:30:00                      0.560750
          2012-01-01 02:00:00                      0.553855

In [8]:
train_df.to_parquet(preprocessed/"selected_blocks_train_val_auto_stat_target.parquet")
joblib.dump(transformer_pipelines, preprocessed/"auto_transformer_pipelines_train_val.pkl")

['data\\london_smart_meters\\preprocessed\\auto_transformer_pipelines_train_val.pkl']